# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [34]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [35]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    #print(vocab)
    vocab_to_int={c: i for i, c in enumerate(vocab)}
    int_to_vocab=dict(enumerate(vocab))
    print("\n", int_to_vocab)
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)


 {0: 'these', 1: 'forget', 2: 'check', 3: 'got', 4: 'me', 5: 'eh', 6: 'to', 7: 'listen', 8: 'little', 9: 'anybody', 10: 'yeah', 11: 'has', 12: 'the', 13: 'only', 14: 'there', 15: "moe's", 16: 'not', 17: 'one', 18: 'days', 19: 'social', 20: 'enhance', 21: 'ice', 22: 'my', 23: 'moe', 24: 'bart_simpson', 25: 'of', 26: 'i', 27: 'on', 28: 'your', 29: 'matter', 30: 'another', 31: 'mike', 32: 'hold', 33: "i'm", 34: 'seen', 35: 'barney_gumble', 36: 'give', 37: 'tavern', 38: 'whats', 39: 'homer', 40: 'catch', 41: 'elite', 42: 'you', 43: 'normal', 44: 'last', 45: 'and', 46: "i'll", 47: 'carve', 48: 'back', 49: 'an', 50: 'drink', 51: 'is', 52: 'where', 53: 'effervescent', 54: 'should', 55: 'skills', 56: 'name', 57: 'lately', 58: 'problems', 59: 'pick', 60: 'hello', 61: 'rotch', 62: 'hey', 63: 'gonna', 64: 'with', 65: "you're", 66: 'homer_simpson', 67: 'meet', 68: 'puke', 69: 'self', 70: 'moe_szyslak'}
Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [36]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation={".":"||Period||", ",":"||Comma||", \
                 "\"":"Quotation_Mark",\
                 ";":"||Semicolon||",\
                 "!":"||Exclamation_Mark||",\
                 "?":"||Question_Mark||",\
                 "(":"||Left_Parentheses||",\
                 ")":"||Right_Parentheses||",\
                 "--":"||Dash||",\
                 "\n":"||Return||"\
                }
    return punctuation

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [341]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)


 {0: 'smell', 1: 'type', 2: 'pass', 3: 'eddie', 4: 'naval', 5: 'decadent', 6: 'past', 7: 'frontrunner', 8: 'solved', 9: "stinkin'", 10: 'funniest', 11: 'chain', 12: 'go', 13: 'nuked', 14: 'purse', 15: 'renee:', 16: 'clench', 17: 'pickle', 18: 'bump', 19: 'dexterous', 20: 'narrator:', 21: 'makes', 22: 'regretted', 23: 'taylor', 24: 'electronic', 25: 'blame', 26: 'five', 27: 'babies', 28: 'inches', 29: 'uglier', 30: 'watch', 31: 'elect', 32: 'tomorrow', 33: 'reality', 34: 'sound', 35: 'we-we-we', 36: "where'd", 37: 'wishes', 38: 'delicious', 39: 'sacajawea', 40: 'windshield', 41: 'penny', 42: 'unintelligent', 43: 'horrible', 44: 'won', 45: 'exchanged', 46: 'but', 47: 'love-matic', 48: 'terror', 49: 'knives', 50: 'this:', 51: 'ivory', 52: 'awful', 53: 'rivalry', 54: 'politician', 55: 'noble', 56: 'ripcord', 57: 'fayed', 58: 'kemi', 59: 'smurfs', 60: 'fruit', 61: 'warn', 62: 'impress', 63: 'sleeps', 64: 'boozebag', 65: 'tanking', 66: 'dory', 67: 'forever', 68: 'hangout', 69: 'like', 70: '

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [342]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [354]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [355]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    targets=tf.placeholder(tf.int32, shape=[None, None], name="targets")
    learning_rate=tf.placeholder(tf.float32, name="learning_rate")
    Input=tf.placeholder(tf.int32, shape=[None, None], name="input")
    return Input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [356]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    basic=tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.tanh, reuse=None)
    basic2=tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.nn.relu, reuse=None)
    #basic3=tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.nn.relu, reuse=None)
    #basic4=tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.nn.relu, reuse=None)
    #basic5=tf.contrib.rnn.BasicLSTMCell(rnn_size, activation=tf.nn.relu, reuse=None)
    
    Cells = tf.contrib.rnn.MultiRNNCell([basic, basic2], state_is_tuple = True) #####

    IS = Cells.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(IS, name="initial_state")
    return Cells, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [357]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    print(input_data, vocab_size, embed_dim)
    inputs = tf.placeholder(tf.int32, [None])
    labels = tf.placeholder(tf.int32, [None, None])
    n_vocab = vocab_size
    n_embedding = embed_dim # Number of embedding features 
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    print("embed:\n",embed)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tensor("Placeholder:0", shape=(50, 5), dtype=int32) 27 256
embed:
 Tensor("embedding_lookup:0", shape=(50, 5, 256), dtype=float32)
Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [358]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """

    # TODO: Implement Function
    #lll = get_init_cell(batch_size, rnn_size)
    Outputs, _state=tf.nn.dynamic_rnn(
    cell,
    inputs,
    dtype = tf.float32,
                                  )
    print(Outputs)
    print()
    print("state:\n", _state)
    print("\n")
    FinalState = tf.identity(_state, name="final_state")
    return Outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tensor("rnn/transpose:0", shape=(?, ?, 256), dtype=float32)

state:
 (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 256) dtype=float32>))


Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [359]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    inputs = get_embed(input_data, vocab_size, embed_dim)
    #print(inputs)
    outputs, fs = build_rnn(cell,inputs)
    #print("outputs:\n",outputs)
    #print("\nfinal_state:\n", final_state)
    #"""
    Logits = tf.contrib.layers.fully_connected(outputs, vocab_size, 
        activation_fn=None, 
        normalizer_fn=None, 
        #normalizer_params=None,
        #weights_initializer=initializers.xavier_initializer(),
        #weights_regularizer=None,
        #biases_initializer=tf.zeros_initializer(),
        #reuse=None,
        trainable=True,
        )
    final_state="test"
    Tuple = (Logits, fs)
    
    #"""
    return Tuple

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tensor("Placeholder:0", shape=(128, 5), dtype=int32) 27 300
embed:
 Tensor("embedding_lookup:0", shape=(128, 5, 300), dtype=float32)
Tensor("rnn/transpose:0", shape=(128, 5, 256), dtype=float32)

state:
 (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 256) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 256) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 256) dtype=float32>))


Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [360]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    print()
    print(len(int_text))
    #print("\nInt_Text\n:", int_text)
    print("batch_size: ", batch_size)
    print()
    print("seq_length: ", seq_length)
    print()
    n_batches = len(int_text)//(batch_size * seq_length)
    print("\nn_batches:\n", n_batches)
    # Only include x's that are not missing y's
    x = np.array(int_text[: n_batches * batch_size * seq_length])
    print("\nx:\n", x)
    y = np.array(int_text[1: (n_batches * batch_size * seq_length) +1])
    print("\ny:\n", y)
    y[-1]=x[0]
    # Reshape and split
    x_batches = np.array_split(x.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.array_split(y.reshape(batch_size, -1), n_batches, 1)
    return np.array(list(zip(x_batches, y_batches)))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)


5000
batch_size:  128

seq_length:  5


n_batches:
 7

x:
 [   0    1    2 ..., 4477 4478 4479]

y:
 [   1    2    3 ..., 4478 4479 4480]
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [393]:
# Number of Epochs
num_epochs = 150
# Batch Size
batch_size = 20
# RNN Size
rnn_size = 300
# Embedding Dimension Size
embed_dim = 50
# Sequence Length
seq_length = 20
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 100

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [394]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

Tensor("input:0", shape=(?, ?), dtype=int32) 6779 50
embed:
 Tensor("embedding_lookup:0", shape=(?, ?, 50), dtype=float32)
Tensor("rnn/transpose:0", shape=(?, ?, 300), dtype=float32)

state:
 (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 300) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 300) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 300) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 300) dtype=float32>))




## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [395]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
global previouslow
batches = get_batches(int_text, batch_size, seq_length)
previouslow =100 #starting loss is actually 8.8 usually. This will ensure that initial value is set on first run.
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))
                currentlow=float(train_loss)
                if float(train_loss) < 1:
                    if currentlow < float(previouslow):
                        previouslow = currentlow
                        itsaver = tf.train.Saver()     
                        itsaver.save(sess, save_dir)
                        print("\nSaved current new low loss of: ", train_loss)

    #Save Model
    #saver = tf.train.Saver()
    #saver.save(sess, save_dir)
    print('Model Trained and Saved')
    print('Saved weights came from when lowest loss was: ',previouslow)


69100
batch_size:  20

seq_length:  20


n_batches:
 172

x:
 [1298 4029 6328 ..., 4443 5817 6269]

y:
 [4029 6328 3761 ..., 5817 6269 2395]
Epoch   0 Batch    0/172   train_loss = 8.821
Epoch   0 Batch  100/172   train_loss = 5.847
Epoch   1 Batch   28/172   train_loss = 5.680
Epoch   1 Batch  128/172   train_loss = 5.244
Epoch   2 Batch   56/172   train_loss = 4.820
Epoch   2 Batch  156/172   train_loss = 4.814
Epoch   3 Batch   84/172   train_loss = 4.760
Epoch   4 Batch   12/172   train_loss = 4.091
Epoch   4 Batch  112/172   train_loss = 4.303
Epoch   5 Batch   40/172   train_loss = 3.900
Epoch   5 Batch  140/172   train_loss = 3.775
Epoch   6 Batch   68/172   train_loss = 3.540
Epoch   6 Batch  168/172   train_loss = 3.230
Epoch   7 Batch   96/172   train_loss = 3.494
Epoch   8 Batch   24/172   train_loss = 3.004
Epoch   8 Batch  124/172   train_loss = 2.857
Epoch   9 Batch   52/172   train_loss = 2.901
Epoch   9 Batch  152/172   train_loss = 2.861
Epoch  10 Batch   80/172   tra

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [396]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [397]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [398]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    print(InputTensor)
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    print(InitialStateTensor)
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    print(FinalStateTensor)
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    global previous
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tensor("input:0", dtype=int32)
Tensor("initial_state:0", dtype=int32)
Tensor("final_state:0", dtype=int32)
Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [399]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    """
    print(str(int_to_vocab[(np.argmax(probabilities))]))
    if str(int_to_vocab[(np.argmax(probabilities))]) == previous:
        probabilities = np.put(probabilities, (np.argmax(probabilities)), 0.0, mode='raise')
        largest = np.argmax(probabilities)
        previous = str(int_to_vocab[largest]        
        else:
            largest = np.argmax(probabilities)
            previous = str(int_to_vocab[largest])
    return str(int_to_vocab[largest])
    ###Tried some experimental IF logic to avoid word repitition.
    """
    
    return str(int_to_vocab[(np.argmax(probabilities))])
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [400]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'barney_gumble' #oe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
Tensor("input:0", shape=(?, ?), dtype=int32)
Tensor("initial_state:0", shape=(2, 2, ?, 300), dtype=float32)
Tensor("final_state:0", shape=(2, 2, ?, 300), dtype=float32)
barney_gumble:(shaking his head) what kind of pathetic drunk do you take me for?(gasp) somebody and it's true. so?
moe_szyslak: ah, i it was unbelievable.


barney_gumble: then to go...(frustrated) no, here you go.
snake_jailbird: you don't see i'm gonna have a middle of the big.
moe_szyslak:(to homer) uh-oh. i'm empty...(bitter) but i quite.
kemi: eh, do are ya..
homer_simpson: well... what do you do--
homer_simpson:(excited gasp) get out of your wife. he's looking a date with her go child...(sobs)
moe_szyslak:(pissed) oh, i admit. my wife was looking for her that happen needed in some bar.
moe_szyslak: get this place a wallet.
moe_szyslak:(defensive) look, that's the good.(explaining) good.


barney_gumble: wow, but i find that, i love a man.
moe_szyslak: ah


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.